In [ ]:
!pip install pandas numpy pyjanitor dateparser pytz ftfy emoji regex rapidfuzz \
             spacy clean-text[gpl] textacy langdetect \
             gensim sentence-transformers bertopic umap-learn hdbscan \
             vaderSentiment \
             networkx cdlib pyvis \
             ydata-profiling
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━

^C


In [ ]:
from __future__ import annotations
import warnings, re
from typing import Optional, List
import pandas as pd
import numpy as np
import janitor
from dateparser import parse as dparse
import ftfy, emoji
import networkx as nx
import spacy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings("ignore")

/usr/local/lib/python3.11/dist-packages/spacy/cli/_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
/usr/local/lib/python3.11/dist-packages/weasel/util/config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


In [ ]:
s_data = pd.read_csv('/content/saved_search_jul7day_ex.csv')

In [ ]:
s_data.head()

,ARTICLE_ID,PUBLICATION,LOAD_DATE,COUNTRY,HEADLINE,ARTICLE TEXT,AUTHOR,SOURCE TYPE,TAGS,CIRCULATION SIZE,...,EDITION,LINK,CUSTOM TAGS,Last Smoke Campaign SENTIMENT,Health Campaign SENTIMENT,Public Health SENTIMENT,World Health Leadership SENTIMENT,State Healthcare Funding SENTIMENT,Penrose Enquiry SENTIMENT,NHS Reform SENTIMENT
0,19679200667,Dakota Broadcasting,2025-07-31,United States,South Dakota DANR announces are quality alert ...,The South Dakota Department of Agriculture and...,uncredited,Regional News,"Air Quality,Climate Change Impacts on Human He...",939,...,NaN,https://dakotabroadcasting.com/south-dakota-da...,NaN,NaN,NaN,Neutral,NaN,NaN,NaN,NaN
1,19679189805,Life Technology,2025-07-31,United States,Study Reveals Statistical Modeling Enhances NT...,Study migrants Italy statistical modeling impr...,uncredited,Consumer,"Google,Google News,Publicis Groupe,PressReader...",2612,...,NaN,https://www.lifetechnology.com/blogs/life-tech...,NaN,NaN,NaN,Positive,NaN,NaN,NaN,NaN
2,19679163086,Htv10 Tv,2025-07-31,United States,"DR. MARLEN J. TRUJILLO, CEO OF SPRING BRANCH C...",Joining the Harris Health Board is a unique ...,uncredited,Regional News,"Harris Health System,Facebook,LinkedIn,Governm...",2886,...,NaN,https://www.htv10.tv/story/52967307/dr-marlen-...,NaN,NaN,NaN,Neutral,NaN,NaN,NaN,NaN
3,19679152080,Archynewsy,2025-07-31,United States,Samcheok City Health Survey 2025: Excellent Area,Samcheok City Recognized for Excellence in R...,Natalie Singh,General News,"Ministry of Health and Prevention (UAE),Corner...",2029,...,NaN,https://www.archynewsy.com/samcheok-city-healt...,NaN,NaN,NaN,Positive,NaN,NaN,NaN,NaN
4,19679132677,ReliefWeb,2025-07-31,United States,Health & Wellness Caseworker,Job Overview: Our Community Health and Well...,uncredited,Government,"Well,IRC,Community Mental Health,Crisis,Famili...",653149,...,NaN,https://reliefweb.int/job/4168098/health-welln...,NaN,NaN,NaN,Positive,NaN,NaN,NaN,NaN


In [ ]:
s_data.columns

Index(['ARTICLE_ID', 'PUBLICATION', 'LOAD_DATE', 'COUNTRY', 'HEADLINE',
       'ARTICLE TEXT', 'AUTHOR', 'SOURCE TYPE', 'TAGS', 'CIRCULATION SIZE',
       'CHANNEL', 'EDITION', 'LINK', 'CUSTOM TAGS',
       'Last Smoke Campaign SENTIMENT', 'Health Campaign SENTIMENT',
       'Public Health SENTIMENT', 'World Health Leadership SENTIMENT',
       'State Healthcare Funding SENTIMENT', 'Penrose Enquiry SENTIMENT',
       'NHS Reform SENTIMENT'],
      dtype='object')

In [ ]:
s_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ARTICLE_ID                          1000 non-null   int64  
 1   PUBLICATION                         1000 non-null   object 
 2   LOAD_DATE                           1000 non-null   object 
 3   COUNTRY                             1000 non-null   object 
 4   HEADLINE                            1000 non-null   object 
 5   ARTICLE TEXT                        1000 non-null   object 
 6   AUTHOR                              1000 non-null   object 
 7   SOURCE TYPE                         1000 non-null   object 
 8   TAGS                                1000 non-null   object 
 9   CIRCULATION SIZE                    1000 non-null   int64  
 10  CHANNEL                             1000 non-null   object 
 11  EDITION                             0 non-nu

In [ ]:
### checking NAs
s_data.isna().sum()

,0
ARTICLE_ID,0
PUBLICATION,0
LOAD_DATE,0
COUNTRY,0
HEADLINE,0
ARTICLE TEXT,0
AUTHOR,0
SOURCE TYPE,0
TAGS,0
CIRCULATION SIZE,0


In [ ]:
#### can drop unnecessary columns such as majority of sentiment columns, Edition, custom tags,

In [ ]:
#### prioritizing the informative column
core_cols = [
    "ARTICLE_ID", "PUBLICATION", "LOAD_DATE", "COUNTRY", "HEADLINE",
    "ARTICLE TEXT", "AUTHOR", "SOURCE TYPE", "TAGS", "CIRCULATION SIZE",
    "CHANNEL", "LINK",
    "Public Health SENTIMENT", "World Health Leadership SENTIMENT",
]

In [ ]:
## saving the dropped columns
DROP_COLS = [
    "EDITION", "CUSTOM TAGS",
    "Last Smoke Campaign SENTIMENT", "Health Campaign SENTIMENT",
    "State Healthcare Funding SENTIMENT", "Penrose Enquiry SENTIMENT",
    "NHS Reform SENTIMENT"
]

In [ ]:
SENT_MAP = {"positive": 1.0, "neutral": 0.0, "negative": -1.0}

****

In [ ]:
### building helpers to preprocess the dataset

In [ ]:
def _to_date(series: pd.Series) -> pd.Series:
    def _p(x):
        if pd.isna(x) or str(x).strip()=="":
            return pd.NaT
        dt = dparse(str(x))
        return pd.NaT if dt is None else pd.Timestamp(dt.date())
    return series.apply(_p)

def _clean_text(s: str) -> str:
    s = ftfy.fix_text(str(s))
    s = emoji.replace_emoji(s, replace="")
    s = re.sub(r"http\S+|\S+@\S+|#\w+|@\w+", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _split_tags(x: str) -> list:
    if not isinstance(x, str) or not x.strip():
        return []
    return [t.strip() for t in x.split(",") if t.strip()]

def _sent_label_to_num(lbl) -> float:
    if not isinstance(lbl, str): return np.nan
    return SENT_MAP.get(lbl.strip().lower(), np.nan)

###Preprocessing

In [ ]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:

    for c in core_cols:
        if c not in df.columns:
            df[c] = np.nan
    df = df[core_cols + [c for c in DROP_COLS if c in df.columns]]
    df = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")


    df = (df
          .remove_empty()
          .drop_duplicates(subset=["ARTICLE_ID"], ignore_index=True))


    df["LOAD_DATE"] = _to_date(df["LOAD_DATE"])
    df["DAY"]   = df["LOAD_DATE"]
    df["WEEK"]  = df["LOAD_DATE"].dt.to_period("W").dt.start_time
    df["MONTH"] = df["LOAD_DATE"].dt.to_period("M").dt.start_time


    df["ARTICLE_TEXT_CLEAN"] = df["ARTICLE TEXT"].astype(str).map(_clean_text)
    df["HEADLINE_CLEAN"]     = df["HEADLINE"].astype(str).map(_clean_text)


    df["TAGS_LIST"] = df["TAGS"].apply(_split_tags)


    df["CIRCULATION_SIZE_NUM"] = pd.to_numeric(df["CIRCULATION SIZE"], errors="coerce")


    for col in ["Public Health SENTIMENT","World Health Leadership SENTIMENT"]:
        df[f"{col}__num"] = df[col].map(_sent_label_to_num)

    def _overall_sent(row):
        ph = row.get("Public Health SENTIMENT__num", np.nan)
        wh = row.get("World Health Leadership SENTIMENT__num", np.nan)
        if pd.notna(ph): return ph
        if pd.notna(wh): return wh
        return np.nan

    df["SENT_OVERALL_NUM"] = df.apply(_overall_sent, axis=1)
    df["SENT_OVERALL_BIN"] = pd.Series(np.select(
        [df["SENT_OVERALL_NUM"]>0.05, df["SENT_OVERALL_NUM"]<-0.05],
        ["Positive","Negative"],
        default="Neutral"
    ))


    df["WORD_LEN"] = df["ARTICLE_TEXT_CLEAN"].str.split().map(len)
    df["CHAR_LEN"] = df["ARTICLE_TEXT_CLEAN"].str.len()

    return df.reset_index(drop=True)

In [ ]:
def derive_conversion_flag(
    df: pd.DataFrame,
    text_keywords: Optional[Iterable[str]] = None,
    tag_includes: Optional[Iterable[str]] = None,
    publication_includes: Optional[Iterable[str]] = None,
    min_circulation_quantile: Optional[float] = None,
    col_name: str = "CONVERSION"
) -> pd.DataFrame:
    conv = np.zeros(len(df), dtype=int)


    if text_keywords:
        pattern = re.compile("|".join([re.escape(k) for k in text_keywords]), flags=re.IGNORECASE)
        mask_text = df["HEADLINE_CLEAN"].str.contains(pattern) | df["ARTICLE_TEXT_CLEAN"].str.contains(pattern)
        conv = np.where(mask_text, 1, conv)


    if tag_includes:
        tag_set = {t.lower() for t in tag_includes}
        mask_tag = df["TAGS_LIST"].apply(lambda lst: any(t.lower() in tag_set for t in lst))
        conv = np.where(mask_tag, 1, conv)


    if publication_includes:
        pub_set = {p.lower() for p in publication_includes}
        mask_pub = df["PUBLICATION"].astype(str).str.lower().isin(pub_set)
        conv = np.where(mask_pub, 1, conv)


    if min_circulation_quantile is not None:
        q = df["CIRCULATION_SIZE_NUM"].quantile(min_circulation_quantile)
        conv = np.where((df["CIRCULATION_SIZE_NUM"] >= q) & (conv==1), 1, 0)

    df[col_name] = conv.astype(int)
    return df

## Exploratory Data Analysis

In [ ]:
def eda_tables(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    out = {}

    out["top_publications"] = (df.groupby("PUBLICATION")["ARTICLE_ID"]
                                 .count().rename("count").reset_index()
                                 .sort_values("count", ascending=False).head(30))
    out["sources_count_avg_circ"] = (
        df.groupby("PUBLICATION", dropna=False)
          .agg(articles=("ARTICLE_ID","count"),
               avg_circ=("CIRCULATION_SIZE_NUM","mean"))
          .reset_index()
          .sort_values(["articles","avg_circ"], ascending=[False, False])
    )
    out["by_source_type"] = (df.groupby("SOURCE TYPE")["ARTICLE_ID"]
                               .count().rename("count").reset_index()
                               .sort_values("count", ascending=False))
    out["by_channel"] = (df.groupby("CHANNEL")["ARTICLE_ID"]
                           .count().rename("count").reset_index()
                           .sort_values("count", ascending=False))
    out["by_country"] = (df.groupby("COUNTRY")["ARTICLE_ID"]
                           .count().rename("count").reset_index()
                           .sort_values("count", ascending=False))

    out["volume_daily"] = (df.groupby("DAY")["ARTICLE_ID"]
                             .count().rename("count").reset_index()
                             .sort_values("DAY"))
    out["volume_weekly"] = (df.groupby("WEEK")["ARTICLE_ID"]
                              .count().rename("count").reset_index()
                              .sort_values("WEEK"))
    out["volume_monthly"] = (df.groupby("MONTH")["ARTICLE_ID"]
                               .count().rename("count").reset_index()
                               .sort_values("MONTH"))

    tags_explode = df[["ARTICLE_ID","TAGS_LIST"]].explode("TAGS_LIST")
    out["top_tags"] = (tags_explode[tags_explode["TAGS_LIST"].notna() & (tags_explode["TAGS_LIST"]!="")]
                         .groupby("TAGS_LIST")["ARTICLE_ID"].count()
                         .rename("count").reset_index()
                         .sort_values("count", ascending=False).head(50))

    out["overall_sentiment"] = (df["SENT_OVERALL_BIN"]
                                .value_counts(dropna=False)
                                .rename_axis("overall_bin")
                                .reset_index(name="count"))
    sent_keep = ["Public Health SENTIMENT","World Health Leadership SENTIMENT"]
    sent_long = df[["ARTICLE_ID"] + sent_keep].melt(
        id_vars=["ARTICLE_ID"], var_name="topic", value_name="label"
    ).dropna(subset=["label"])
    out["sentiment_by_topic"] = (sent_long.groupby(["topic","label"])["ARTICLE_ID"]
                                 .count().rename("count").reset_index()
                                 .sort_values(["topic","count"], ascending=[True, False]))

    if "CONVERSION" in df.columns:
        conv_rate = pd.DataFrame({
            "metric":["total_articles","conversion_articles","conversion_rate"],
            "value":[len(df), int(df["CONVERSION"].sum()),
                     (df["CONVERSION"].sum()/len(df) if len(df) else np.nan)]
        })
        out["conversion_summary"] = conv_rate
    return out

In [ ]:
def keywords_high_circ_conversion(
    df: pd.DataFrame,
    conv_col: str = "CONVERSION",
    circ_col: str = "CIRCULATION_SIZE_NUM",
    text_col: str = "ARTICLE_TEXT_CLEAN",
    quantile: float = 0.75,
    n_terms: int = 20,
    ngram_range: Tuple[int,int] = (1,2),
) -> pd.DataFrame:
    if conv_col in df.columns and df[conv_col].sum() > 0:
        subset = df[(df[conv_col]==1)]
    else:
        subset = df.copy()

    q = subset[circ_col].quantile(quantile)
    subset = subset[subset[circ_col] >= q]
    corpus = subset[text_col].fillna("").tolist()
    if not corpus:
        return pd.DataFrame(columns=["term","count"])

    vec = CountVectorizer(lowercase=True, stop_words="english",
                          max_df=0.9, min_df=2, ngram_range=ngram_range)
    X = vec.fit_transform(corpus)
    counts = np.asarray(X.sum(axis=0)).ravel()
    terms = np.array(vec.get_feature_names_out())
    order = counts.argsort()[::-1][:n_terms]
    return pd.DataFrame({"term": terms[order], "count": counts[order]})

****

### Network Analysis Tables

In [ ]:
def eda_network(df: pd.DataFrame, sample_n: int = 2000, min_cooccurrences: int = 2):
    n = min(sample_n, len(df))
    texts = df["ARTICLE_TEXT_CLEAN"].head(n).tolist()
    nlp = spacy.load("en_core_web_sm", disable=["lemmatizer"])
    G = nx.Graph()
    for t in texts:
        doc = nlp(t[:2000])
        ents = list({e.text.strip() for e in doc.ents if e.label_ in {"PERSON","ORG"}})
        for i in range(len(ents)):
            for j in range(i+1, len(ents)):
                a, b = sorted([ents[i], ents[j]])
                if G.has_edge(a,b):
                    G[a][b]["weight"] += 1
                else:
                    G.add_edge(a,b, weight=1)

    to_remove = [(u,v) for u,v,w in G.edges(data="weight") if w < min_cooccurrences]
    G.remove_edges_from(to_remove)
    G.remove_nodes_from([n for n in list(G.nodes) if G.degree(n)==0])


    if G.number_of_nodes()==0:
        summary = pd.DataFrame(columns=["node","degree","betweenness"])
    else:
        deg = dict(G.degree())
        bet = nx.betweenness_centrality(G)
        summary = (pd.DataFrame({"node": list(G.nodes),
                                 "degree": [deg[n] for n in G.nodes],
                                 "betweenness": [bet[n] for n in G.nodes]})
                   .sort_values(["betweenness","degree"], ascending=False)
                   .reset_index(drop=True))
    return G, summary

In [ ]:
def plot_network_clean(G, net_summary, top_n=40, label_top=12, edge_keep_pct=50, savepath="network_top40_clean.png"):
    if G is None or net_summary is None or net_summary.empty or G.number_of_nodes()==0:
        return
    top_nodes = net_summary.head(top_n)["node"].tolist()
    SG = G.subgraph(top_nodes).copy()
    if SG.number_of_nodes()==0: return

    comps = list(nx.connected_components(SG))
    if not comps: return
    SG = SG.subgraph(max(comps, key=len)).copy()
    if SG.number_of_edges()==0: return

    weights = np.array([SG[u][v].get("weight",1.0) for u,v in SG.edges()])
    thresh = np.percentile(weights, 100 - edge_keep_pct)  # keep top X%
    SG.remove_edges_from([(u,v) for u,v in SG.edges() if SG[u][v].get("weight",1.0) < thresh])
    SG.remove_nodes_from([n for n in list(SG.nodes) if SG.degree(n)==0])
    if SG.number_of_nodes()==0: return

    pos = nx.spring_layout(SG, seed=42, iterations=200)
    bet = nx.betweenness_centrality(SG)
    ranked = sorted(SG.nodes(), key=lambda n: bet.get(n,0.0), reverse=True)
    sizes_map = {n:s for n,s in zip(ranked, np.linspace(400,2200,num=len(ranked)))}
    sizes = [sizes_map[n] for n in SG.nodes()]
    ewidths = [0.6 + np.log1p(SG[u][v].get("weight",1.0)) for u,v in SG.edges()]

    label_nodes = set(ranked[:label_top])
    labels = {n:n for n in SG.nodes() if n in label_nodes}

    plt.figure(figsize=(10,8))
    nx.draw_networkx_edges(SG, pos, width=ewidths, alpha=0.5)
    nx.draw_networkx_nodes(SG, pos, node_size=sizes)
    nx.draw_networkx_labels(SG, pos, labels=labels, font_size=8)
    plt.title(f"Entity Co-occurrence Network (Top {min(top_n, len(net_summary))}, cleaned)")
    plt.axis("off"); plt.tight_layout(); plt.savefig(savepath, dpi=150); plt.close()

In [ ]:
def _rotate(ax, deg=45):
    for t in ax.get_xticklabels(): t.set_rotation(deg); t.set_ha("right")

def plot_bar(df: pd.DataFrame, x: str, y: str, title: str, xlabel: str, ylabel: str, fname: str, horizontal=False):
    if isinstance(df, dict):
        df = pd.DataFrame(df)
    if df is None or df.empty: return
    plt.figure()
    if horizontal:
        plt.barh(df[x][::-1], df[y][::-1])
    else:
        plt.bar(df[x], df[y])
        _rotate(plt.gca(), 45)
    plt.title(title); plt.xlabel(xlabel); plt.ylabel(ylabel)
    plt.tight_layout(); plt.savefig(fname, dpi=150); plt.close()

def plot_line(df: pd.DataFrame, x: str, y: str, title: str, xlabel: str, ylabel: str, fname: str):
    if df is None or df.empty: return
    plt.figure()
    plt.plot(df[x], df[y])
    plt.title(title); plt.xlabel(xlabel); plt.ylabel(ylabel)
    _rotate(plt.gca(), 45); plt.tight_layout(); plt.savefig(fname, dpi=150); plt.close()

In [ ]:
def _clean_text(s) -> str:
    try:
        s = "" if s is None or (isinstance(s, float) and pd.isna(s)) else str(s)
        s = ftfy.fix_text(s)
        s = emoji.replace_emoji(s, replace="")
        s = re.sub(r"http\S+|\S+@\S+|#\w+|@\w+", " ", s)
        s = re.sub(r"\s+", " ", s).strip()
        return s
    except Exception:
        return ""

In [ ]:
df_clean = preprocess(s_data)
tables = eda_tables(df_clean)
kw_conv = keywords_high_circ_conversion(df_clean, conv_col="CONVERSION")

****

### EDA Visuals of EDA Tables

In [ ]:
G, net_summary = eda_network(df_clean)
plot_bar(tables["top_publications"].head(15), "PUBLICATION","count",
         "Top Publications by Article Count","Publication","Count","eda_top_publications.png", horizontal=True)

plot_bar(tables["sources_count_avg_circ"].head(20), "PUBLICATION","avg_circ",
         "Top Publications by Average Circulation","Publication","Avg Circulation","eda_sources_avg_circ.png", horizontal=True)

plot_bar(tables["by_source_type"], "SOURCE TYPE","count",
         "Items by Source Type","Source Type","Count","eda_by_source_type.png")

plot_bar(tables["by_channel"], "CHANNEL","count",
         "Items by Channel","Channel","Count","eda_by_channel.png")

plot_line(tables["volume_daily"], "DAY","count","Daily Volume","Day","Count","eda_volume_daily.png")
plot_line(tables["volume_weekly"], "WEEK","count","Weekly Volume","Week","Count","eda_volume_weekly.png")
plot_line(tables["volume_monthly"], "MONTH","count","Monthly Volume","Month","Count","eda_volume_monthly.png")

plot_bar(tables["overall_sentiment"], "overall_bin","count",
         "Overall Sentiment (Proxy)","Sentiment","Count","eda_overall_sentiment.png")

plot_bar(tables["sentiment_by_topic"], "topic","count",
         "Sentiment-Tagged Topics (Total Volume)","Topic","Count","eda_sentiment_by_topic.png", horizontal=True)

plot_bar(kw_conv, "term","count",
         "Top Keywords: High-Circulation Conversion Articles","Term","Count","eda_keywords_high_circ_conv.png", horizontal=True)

plot_network_clean(G, net_summary, top_n=40, label_top=12, edge_keep_pct=50, savepath="network_top40_clean.png")

print("Saved figures:")
for f in [
    "eda_top_publications.png","eda_sources_avg_circ.png",
    "eda_by_source_type.png","eda_by_channel.png",
    "eda_volume_daily.png","eda_volume_weekly.png","eda_volume_monthly.png",
    "eda_overall_sentiment.png","eda_sentiment_by_topic.png",
    "eda_keywords_high_circ_conv.png","network_top40_clean.png"
]:
    print("-", f)

Saved figures:
- eda_top_publications.png
- eda_sources_avg_circ.png
- eda_by_source_type.png
- eda_by_channel.png
- eda_volume_daily.png
- eda_volume_weekly.png
- eda_volume_monthly.png
- eda_overall_sentiment.png
- eda_sentiment_by_topic.png
- eda_keywords_high_circ_conv.png
- network_top40_clean.png
